# Gemini API: Function calling with Python

Date: 2024/12/26

Reference: https://github.com/google-gemini/cookbook/blob/main/quickstarts/Function_calling.ipynb


In [1]:
!pip install -U -q "google-generativeai>=0.7.2"  # Install the Python SDK


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import google.generativeai as genai

/Users/shiny/Documents/GitHub/node-red-genai/etc/ai_agents/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import os

GOOGLE_API_KEY = os.environ["GOOGLE_AI_API_KEY"]
genai.configure(api_key=GOOGLE_API_KEY)

## Function calling basics

In [9]:
def add(a: float, b: float):
    """returns a + b."""
    return a + b


def subtract(a: float, b: float):
    """returns a - b."""
    return a - b


def multiply(a: float, b: float):
    """returns a * b."""
    return a * b


def divide(a: float, b: float):
    """returns a / b."""
    return a / b


model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", tools=[add, subtract, multiply, divide]
)

model

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x12483d1d0>,
    system_instruction=None,
    cached_content=None
)

## Automatic function calling

In [10]:
chat = model.start_chat(enable_automatic_function_calling=True)

In [11]:
response = chat.send_message(
    "I have 57 cats, each owns 44 mittens, how many mittens is that in total?"
)
response.text

'57 cats * 44 mittens/cat = 2508 mittens in total.\n'

In [12]:
57 * 44

2508

In [13]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 80)

user -> [{'text': 'I have 57 cats, each owns 44 mittens, how many mittens is that in total?'}]
--------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}]
--------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}]
--------------------------------------------------------------------------------
model -> [{'text': '57 cats * 44 mittens/cat = 2508 mittens in total.\n'}]
--------------------------------------------------------------------------------


## Manual function calling

In [15]:
def find_movies(description: str, location: str = ""):
    """find movie titles currently playing in theaters based on any description, genre, title words, etc.

    Args:
        description: Any kind of description including category or genre, title words, attributes, etc.
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
    """
    return ["Barbie", "Oppenheimer"]


def find_theaters(location: str, movie: str = ""):
    """Find theaters based on location and optionally movie title which are is currently playing in theaters.

    Args:
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
        movie: Any movie title
    """
    return ["Googleplex 16", "Android Theatre"]


def get_showtimes(location: str, movie: str, theater: str, date: str):
    """
    Find the start times for movies playing in a specific theater.

    Args:
      location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
      movie: Any movie title
      thearer: Name of the theater
      date: Date for requested showtime
    """
    return ["10:00", "11:00"]
     

In [16]:
functions = {
    "find_movies": find_movies,
    "find_theaters": find_theaters,
    "get_showtimes": get_showtimes,
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=functions.values())

In [43]:
response = model.generate_content(
    "Which theaters in Mountain View show the Barbie movie?"
)
part = response.candidates[0].content.parts[0]

In [44]:
part.function_call

name: "find_theaters"
args {
  fields {
    key: "movie"
    value {
      string_value: "Barbie"
    }
  }
  fields {
    key: "location"
    value {
      string_value: "Mountain View"
    }
  }
}

In [45]:
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    return functions[function_name](**function_args)


part = response.candidates[0].content.parts[0]

# Check if it's a function call; in real use you'd need to also handle text
# responses as you won't know what the model will respond with.
if part.function_call:
    result = call_function(part.function_call, functions)

print(result)

['Googleplex 16', 'Android Theatre']
